In [26]:
%pip install pandas
%pip install prophet
%pip install holidays
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [27]:
from prophet import Prophet
import pandas as pd
import numpy as np
import pickle

# Data Preparation

In [28]:
df_bills = pd.read_csv('data/bills.csv')
df_venues = pd.read_csv('data/venues.csv')

In [29]:
df_venues.columns

Index(['venue_xref_id', 'concept', 'city', 'country', 'start_of_day_offset'], dtype='object')

In [30]:
df_venues = df_venues.drop(columns='start_of_day_offset')
df_venues

,venue_xref_id,concept,city,country
0,ea74268311cfcc47d2b4c38ef08b9dab0117226a9a99a9...,FAST_FOOD,Albuquerque,US
1,e7cec41c46a9706ba1a702b368be5431913b6dc9c9da49...,FAMILY_DINING,Brooklyn,US
2,e63e79791883b0c4ac71d41eaee727932c13d6a4ec8f78...,FAST_CASUAL,Windsor,CA
3,c100388c9328b30b6bdfbea43f113d367ae9d32b03e84b...,BAR,Burnaby,CA
4,a819ee7f83f27dd3d7f44515e22aca3c26e6704ddd2a9b...,FAMILY_DINING,Tisdale,CA
...,...,...,...,...
596,cfd1fa6e8f122e93b1cb02e7b7541749335e7088095f8e...,NaN,Toronto,CA
597,75e7251298f2d62406dbfe1a11011cffdad90b27704687...,NaN,Richardson,US
598,40c38939612f59ad418569e94a075935576bff0449c7f8...,FAST_CASUAL,Guelph,CA
599,2ed5bf26505a6064d09b40bacdb6fccc9665c3de029494...,NaN,Mississauga,CA


## Data Cleaning

In [31]:
missing_venues = df_venues[~df_venues['venue_xref_id'].isin(df_bills['venue_xref_id'])]
missing_venues

# All 'venue_xref_id' in df_venues exist in 'venue_xref_id' column in df_bills

,venue_xref_id,concept,city,country


In [32]:
print(df_venues['concept'].unique())

# We will omit fine_dining restaurants since they don't do promotions
# We will omit pop_up restaurants since they don't have long-term data
concept_counts = df_venues['concept'].value_counts(dropna=False)
filtered_counts = concept_counts.loc[['FINE_DINING', 'POP_UP']]
nan_count = df_venues['concept'].isna().sum()
filtered_counts.loc['NaN'] = nan_count
print(filtered_counts)

['FAST_FOOD' 'FAMILY_DINING' 'FAST_CASUAL' 'BAR' 'CAFE' 'SPORTS_CLUB' nan
 'FINE_DINING' 'BREWERY' 'POP_UP' 'BAKERY' 'BUFFET' 'HOTEL'
 'ENTERTAINMENT_COMPLEX' 'FOOD_TRUCK']
concept
FINE_DINING     19
POP_UP           1
NaN            136
Name: count, dtype: int64


In [33]:
print(df_venues['city'].isna().sum())
# No NaN in city column

0


In [34]:
# Get venue_xref_id for rows where concept is 'FINE_DINING', 'POP_UP', or NaN
venues_to_remove = df_venues[df_venues['concept'].isin(['FINE_DINING', 'POP_UP']) | df_venues['concept'].isna()]['venue_xref_id'].unique()
# Keep only rows where venue_xref_id is NOT in venues_to_remove
df_venues = df_venues[~df_venues['venue_xref_id'].isin(venues_to_remove)]
# Remove rows from df_bills that have the filtered venue_xref_id
df_bills = df_bills[~df_bills['venue_xref_id'].isin(venues_to_remove)]
print(f"Filtered df_venues: {df_venues.shape}")
print(f"Filtered df_bills: {df_bills.shape}")


Filtered df_venues: (445, 4)
Filtered df_bills: (7121732, 20)


In [35]:
df_bills.columns

Index(['bill_paid_at_local', 'bill_total_billed',
       'bill_total_discount_item_level', 'bill_total_gratuity',
       'bill_total_net', 'bill_total_tax', 'bill_total_voided', 'bill_uuid',
       'business_date', 'order_duration_seconds', 'order_seated_at_local',
       'order_closed_at_local', 'order_take_out_type_label', 'order_uuid',
       'payment_amount', 'payment_count', 'payment_total_tip',
       'sales_revenue_with_tax', 'venue_xref_id', 'waiter_uuid'],
      dtype='object')

In [36]:
df_merged = df_bills.merge(df_venues, on="venue_xref_id", how="left")
df_merged


,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,...,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid,concept,city,country
0,2024-07-02 20:00:53,102.53,0.00,0.0,95.00,7.53,0.0,240702200053~8792ADCC-545E-4AF3-9836-9C428ED8285C,2024-07-02,5071,...,240702183622~4B0A3D27-501D-4C94-BEBC-7B7F3C91A0F4,124.53,1,22.0,102.53,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,200421150750~37D0C51E-EC4F-4EA1-B549-D223DA183ABD,BAR,Grand Forks,US
1,2024-07-03 21:43:39,5.38,5.00,0.0,5.00,0.38,0.0,240703214339~B06B2A2F-B1A0-44C0-920A-242F41B58BD2,2024-07-03,6968,...,240703194731~40651D6D-4A16-4889-8C12-0F9A00D8BCF7,5.38,1,0.0,5.38,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
2,2024-07-03 18:23:56,141.90,0.00,0.0,132.00,9.90,0.0,240703182356~D082FE98-841C-4EAB-B15E-BEA41FA1CDF0,2024-07-03,2011,...,240703175025~0AEB61B0-220A-45A3-807E-78A1F432BB4C,171.90,1,30.0,141.90,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
3,2024-07-03 19:33:21,63.77,0.00,0.0,59.00,4.77,0.0,240703193321~273CBB07-FAB0-49B7-A445-8474F9A4570F,2024-07-03,5480,...,240703180201~4B276496-BE92-4DE6-BD75-565929827014,73.77,1,10.0,63.77,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
4,2024-07-05 21:01:14,210.65,0.00,0.0,195.00,15.65,0.0,240705210114~D423B906-1AEE-4DB8-84BA-234267A14E05,2024-07-05,4773,...,240705194141~F90FEB4F-D238-449E-B476-D16966DF4EEC,252.65,1,42.0,210.65,885332b7f22a142e21b7459473003fddc17bfca5753ceb...,220104154507~89085BB4-771B-40A1-9D20-B9745CC1DAFA,BAR,Grand Forks,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7121727,2024-12-29 20:32:54,82.42,0.00,0.0,82.42,0.00,0.0,241229203254~464AA06D-F986-4C51-8A53-A7B8362072D6,2024-12-29,4382,...,241229191952~96F91DE2-5ECC-4AC9-9228-B0741F3494B1,82.42,1,0.0,82.42,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,220831171122~1455DBA8-211C-4281-BF27-8698365CA3EB,FAMILY_DINING,Edmonton,CA
7121728,2024-12-30 20:36:19,87.92,0.00,0.0,87.92,0.00,0.0,241230203619~37348A86-98D1-4113-8EDD-06A3CF61A85B,2024-12-30,5244,...,241230190855~BB4604F2-B7BA-487A-99AF-7AEB77CA3EEC,87.92,1,0.0,87.92,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6,FAMILY_DINING,Edmonton,CA
7121729,2024-12-31 14:24:23,51.96,0.00,0.0,51.96,0.00,0.0,241231142423~9D2E9295-9E59-4E76-8242-D230C9FD2494,2024-12-31,963,...,241231140820~4D531668-3D68-4789-A1DB-BE324D4583D5,51.96,1,0.0,51.96,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,161216233231~64B016BA-D9BA-49C7-8726-4BCB6516A11D,FAMILY_DINING,Edmonton,CA
7121730,2024-12-31 17:00:34,49.35,11.59,0.0,49.35,0.00,0.0,241231170034~A9B4401D-26DC-4FC5-A4F9-0261D950DB3B,2024-12-31,59,...,241231165935~27989697-086C-4F7C-8AB2-8C0A8B2CF3A7,49.35,1,0.0,49.35,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6,FAMILY_DINING,Edmonton,CA


In [37]:
df_merged['order_take_out_type_label'].unique()
# 'dinein' = 0, 'takeout' = 1, 'delivery' = 2, 'bartab' = 3, 'onlineorder' = 4

array(['dinein', 'takeout', 'delivery', 'bartab', 'onlineorder'],
      dtype=object)

In [38]:
import holidays
import numpy as np
import pandas as pd

def add_columns(df):
    holiday_calendars = {
        'US': holidays.US(),
        'CA': holidays.CA()
    }

    # Ensure bill_paid_at_local is in datetime format
    df['bill_paid_at_local'] = pd.to_datetime(df['bill_paid_at_local'], errors='coerce')

    def is_holiday(row):
        country_code = row['country']
        date = row['bill_paid_at_local']
        
        # Default to 0 if country is missing or not in the dictionary
        if country_code not in holiday_calendars or pd.isna(date):
            return 0

        return 1 if date in holiday_calendars[country_code] else 0
    
    df['order_type_encoded'] = df['order_take_out_type_label'].astype('category').cat.codes
    df['holiday'] = df.apply(is_holiday, axis=1)
    df['time'] = df['bill_paid_at_local'].dt.strftime('%H:%M:%S')  # Extract time as string
    df['day_of_week'] = df['bill_paid_at_local'].dt.dayofweek  # Monday=0, Sunday=6
    df['hour_of_day'] = df['bill_paid_at_local'].dt.hour
    df['is_weekend'] = df['bill_paid_at_local'].dt.dayofweek.isin([5, 6]).astype(int)

    df.rename(columns={'payment_count': 'num_people'}, inplace=True)

    # Avoid division errors: Fill NaN values and avoid division by zero
    df['payment_per_person'] = df['payment_amount'] / df['num_people'].replace(0, np.nan)
    df['payment_per_person'] = df['payment_per_person'].fillna(0)

    return df

df = add_columns(df_merged)
df


,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,...,concept,city,country,order_type_encoded,holiday,time,day_of_week,hour_of_day,is_weekend,payment_per_person
0,2024-07-02 20:00:53,102.53,0.00,0.0,95.00,7.53,0.0,240702200053~8792ADCC-545E-4AF3-9836-9C428ED8285C,2024-07-02,5071,...,BAR,Grand Forks,US,2,0,20:00:53,1,20,0,124.53
1,2024-07-03 21:43:39,5.38,5.00,0.0,5.00,0.38,0.0,240703214339~B06B2A2F-B1A0-44C0-920A-242F41B58BD2,2024-07-03,6968,...,BAR,Grand Forks,US,2,0,21:43:39,2,21,0,5.38
2,2024-07-03 18:23:56,141.90,0.00,0.0,132.00,9.90,0.0,240703182356~D082FE98-841C-4EAB-B15E-BEA41FA1CDF0,2024-07-03,2011,...,BAR,Grand Forks,US,2,0,18:23:56,2,18,0,171.90
3,2024-07-03 19:33:21,63.77,0.00,0.0,59.00,4.77,0.0,240703193321~273CBB07-FAB0-49B7-A445-8474F9A4570F,2024-07-03,5480,...,BAR,Grand Forks,US,2,0,19:33:21,2,19,0,73.77
4,2024-07-05 21:01:14,210.65,0.00,0.0,195.00,15.65,0.0,240705210114~D423B906-1AEE-4DB8-84BA-234267A14E05,2024-07-05,4773,...,BAR,Grand Forks,US,2,0,21:01:14,4,21,0,252.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7121727,2024-12-29 20:32:54,82.42,0.00,0.0,82.42,0.00,0.0,241229203254~464AA06D-F986-4C51-8A53-A7B8362072D6,2024-12-29,4382,...,FAMILY_DINING,Edmonton,CA,2,0,20:32:54,6,20,1,82.42
7121728,2024-12-30 20:36:19,87.92,0.00,0.0,87.92,0.00,0.0,241230203619~37348A86-98D1-4113-8EDD-06A3CF61A85B,2024-12-30,5244,...,FAMILY_DINING,Edmonton,CA,2,0,20:36:19,0,20,0,87.92
7121729,2024-12-31 14:24:23,51.96,0.00,0.0,51.96,0.00,0.0,241231142423~9D2E9295-9E59-4E76-8242-D230C9FD2494,2024-12-31,963,...,FAMILY_DINING,Edmonton,CA,4,0,14:24:23,1,14,0,51.96
7121730,2024-12-31 17:00:34,49.35,11.59,0.0,49.35,0.00,0.0,241231170034~A9B4401D-26DC-4FC5-A4F9-0261D950DB3B,2024-12-31,59,...,FAMILY_DINING,Edmonton,CA,4,0,17:00:34,1,17,0,49.35


In [39]:
# Extract test df and train df
test_venues = df['venue_xref_id'].drop_duplicates().head(10).values
df_test = df[df['venue_xref_id'].isin(test_venues)].copy()
df_train = df[~df['venue_xref_id'].isin(test_venues)].copy()

df_test.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

print(f"Original DataFrame size: {df.shape}")
print(f"Training DataFrame size: {df_train.shape}")
print(f"Test DataFrame size: {df_test.shape}")


Original DataFrame size: (7121732, 30)
Training DataFrame size: (6954315, 30)
Test DataFrame size: (167417, 30)


# ML Prediction Model

## Earning Prediction Model

In [40]:
df_ep = df_train[['bill_paid_at_local', 'bill_total_net', 'bill_total_billed', 'order_type_encoded', 
                         'payment_amount', 'num_people', 'holiday', 'time', 'day_of_week', 
                         'hour_of_day', 'is_weekend', 'payment_per_person']].rename(
    columns={'bill_paid_at_local': 'ds', 'bill_total_net': 'y'}
)

df_ep['ds'] = pd.to_datetime(df_ep['ds'])

In [ ]:
prophet_earnings = Prophet()

extra_features = ['bill_total_billed', 'order_type_encoded', 'payment_amount', 'num_people', 
                  'holiday', 'day_of_week', 'hour_of_day', 'is_weekend', 'payment_per_person']

for feature in extra_features:
    prophet_earnings.add_regressor(feature)

prophet_earnings.fit(df_ep)

with open('prophet_earnings_model.pkl', 'wb') as f:
    pickle.dump(prophet_earnings, f)

with open('prophet_earnings_model.pkl', 'rb') as f:
    prophet_earnings = pickle.load(f)

## Potential Earning Prediction Model